# Quantum Operations

Each set of quantum operations defines operations for a particular type of qubit.
At the moment the library only provides operations for tunable transmon qubits.
We'll introduce you to these operations and show you how to add to or modify them.
You can also create your own kind of qubit and quantum operations for them but that
will not be covered in this tutorial.

Let's get started.

## Setting up a device and session

Build your LabOne Q `DeviceSetup`, qubits and `Session` as normal. Here we import an example from the applications library's test suite (this will change in the near future):

In [ ]:
from laboneq.simple import *

from laboneq_applications.qpu_types.tunable_transmon import demo_platform

In [ ]:
# Create a demonstration QuantumPlatform for a tunable-transmon QPU:
qt_platform = demo_platform(n_qubits=6)

# The platform contains a setup, which is an ordinary LabOne Q DeviceSetup:
setup = qt_platform.setup

# And a tunable-transmon QPU:
qpu = qt_platform.qpu

# Inside the QPU, we have qubits, which is a list of six LabOne Q Application
# Library TunableTransmonQubit qubits:
qubits = qpu.qubits

In [ ]:
session = Session(setup)
session.connect(do_emulation=True)

## Qubits and qubit parameters

Inspect the qubit parameters

In [ ]:
print(qubits[0].parameters)

The following qubit parameters are used by the Applications Library:

* Parameters with the prefixes `ge_drive_`/`ef_drive_` are used to configure the parameters for implementing a pi-pulse on the ge and ef transitions.
* Parameters with the prefix `readout_` are used to configure the parameters of the readout pulse.
* Parameters with the prefix `readout_integration_` are used to configure the parameters of the integration kernels. Setting the parameter `readout_integration_kernels=default` indicates that a constant square pulse with the length given by `readout_integration_length` will be used for the integration (created in `qubit.default_integration_kernels()`). The parameter `readout_integration_kernels` can also be set to a list of pulse dictionaries of the form `{"function": pulse_functional_name, "func_par1": value, "func_par2": value, ... }`. `pulse_functional_name` must be the name of a function registered with the `pulse_library.register_pulse_functional` [decorator](https://docs.zhinst.com/labone_q_user_manual/tutorials/reference/04_pulse_library/).
* `reset_delay_length`: the waiting time for passive qubit reset.
* `resonance_frequency_ge`, `resonance_frequency_ef` ' `drive_lo_frequency`, `readout_resonator_frequency`, `readout_lo_frequency`, `drive_range`, `readout_range_out`, `readout_range_in` are used to configure the qubit calibration which then ends up in the `Experiment` calibration.

The remaining qubit parameters are still there for legacy reasons and have no effect. These will be cleaned up soon.

## Quantum Operations

Quantum operations provide a means for writing DSL at a higher level of abstraction than in base LabOne Q. When writing LabOne Q DSL one works with operations on signal lines. When writing DSL with quantum operations, one works with operations on *qubits*.

**Note**:

The experiments built using quantum operations are just ordinary LabOne Q experiments. It's how the experiments are described that differs. One also uses LabOne Q DSL to *define* quantum operations and one can combine quantum operations with ordinary LabOne Q DSL, because they are producing the same DSL.

### Building a first experiment pulse sequence

Let's build our first experiment pulse sequence using quantum operations. The experiment pulse sequence is described by the LabOne Q `Experiment` object.

We'll need to import some things are the start. We'll explain what each of them is as we go:

In [ ]:
import numpy as np

from laboneq_applications import dsl
from laboneq_applications.qpu_types.tunable_transmon import (
    TunableTransmonOperations,
)

Let's start with a tiny experiment sequence that rotates a qubit a given angle about the x-axis and performs a measurement:

In [ ]:
@dsl.qubit_experiment
def rotate_and_measure(qop, q, angle, count=10):
    """Rotate q by the given angle and measure it."""
    with dsl.acquire_loop_rt(count=count):
        qop.rx(q, angle)
        qop.measure(q, "measure_q")

and break down the code line by line:

* `@dsl.qubit_experiment`: This decorator creates a new experiment object and makes it accessible inside the `rotate_and_measure` function. It also finds the qubits in the function arguments (i.e. `q`) and sets the experiment calibration using them.

* `def rotate_and_measure(qop, q, angle, count=10):`: These are ordinary function arguments, except for the detection of the qubit objects just mentioned. The `qop` argument supplies the set of quantum operations to use. The same function can be used to build an experiment for any qubit platform that provides the same operations.

* `with dsl.acquire_loop_rt(count=count)`: This is just the `acquire_loop_rt` function from `laboneq.dsl.experiments.builtins`. The `laboneq_applications.dsl` module is just a convenient way to access the LabOne Q DSL functionality.

* `qop.rx(q, angle)`: Here `qop` is a set of quantum operations. The `rx` operation creates a pulse that rotates the qubit by the given angle (in radians) by linearly scaling the pulse amplitude with respect to the qubit pi-pulse amplitude stored in `qubit.parameters.drive_parameters_ge.amplitdue_pi`. The pulse type is specified in `qubit.parameters.drive_parameters_ge.pulse.function` and it uses the length in `qubit.parameters.drive_parameters_ge.length`.
   * To implement a pi-pulse and a pi-half pulse, we provide the operations `qop.x180`, `qop.y180`, `qop.x90`, `qop.y90`, which use the pulse amplitdues values in `qubit.parameters.drive_parameters_ge.amplitdue_pi` and `qubit.parameters.drive_parameters_ge.amplitdue_pi2`, respectively,

* `qop.measure(q, "measure_q")`: Performs a measurement on the qubit, using the readout pulse and kernels specified by the qubit parameters `qubit.parameters.readout_parameters` and `qubit.parameters.readout_integration_parameters`. `"measure_q"` is the handle to store the results under.

To build the experiment we need some qubits and a set of quantum operations. Let's use the `TunableTransmonOperations` provided by the applications library and the qubit we defined earlier:

In [ ]:
qop = TunableTransmonOperations()
q0 = qubits[0]

exp = rotate_and_measure(qop, q0, np.pi / 2, count=10)

Here `exp` is just an ordinary LabOne Q experiment:

In [ ]:
print(exp)

Have a look through the generated experiment and check that:

* the experiment signals are those for the qubit.
* the qubit calibration has been set.
* the experiment sections are those you expect.

### Examining the set of operations

So far we've treated the quantum operations as a black box. Now let's look inside. We can start by listing the quantum operations:

In [ ]:
qop.keys()

The quantum operations have an attribute `QUBIT_TYPES` which specifies the type of qubits support by the quantum operations object we've created. In our case, that's the `TunableTransmonQubit`:

In [ ]:
qop.QUBIT_TYPES

Under the hood there is a `BASE_OPS` attribute. This is an implementation detail -- it contains the original definitions of the quantum operations. We will ignore it for now except to mention that individual quantum operations can be overridden with alternative implementations if required.

Let's take a look at one of the quantum operations.

### Working with a quantum operation

In [ ]:
qop.rx?

In [ ]:
qop.rx.src

One can write:

* `qop.rx?` to view the documentation as usual, or
* `qop.rx.src` to easily see how a quantum operation is implemented.

Take a moment to read the documentation of a few of the other operations and their source, for example:

In [ ]:
qop.x180.src

In [ ]:
qop.x90.src

Calling a quantum operation by itself produces a LabOne Q section:

In [ ]:
section = qop.rx(qubits[0], np.pi)
print(section)

Some things to note about the section:

* The section name is the name of the quantum operation, followed by the UIDs of the qubits it is applied to.
* The section UID is automatically generated from the name.
* The section starts by reserving all the signal lines of the qubit it operates on so that operations acting on the same qubits never overlap.

In addition to `.src` each quantum operation also has two special attributes:

* `.op`: This returns the function that implements the quantum operation.
* `.omit_section(...)`: This method builds the quantum operation but without a containing section. This is useful if one wants to define a quantum operation in terms of another, but not have deeply nested sections.

Let's look at `.op` now. We'll use `.omit_section` once we've seen how to write our own operations.

In [ ]:
qop.rx.op

### Writing a quantum operation

Often you'll want to write your own quantum operation, either to create a new operation or to replace an existing one.

Let's write our own very simple implementation of an `rx` operation that varies the pulse length instead of the amplitude:

In [ ]:
@qop.register
def simple_rx(self, q, angle):
    """A very simple implementation of an RX operation that varies pulse length."""
    # Determined via rigorously calibration ;) :
    amplitude = 0.6
    length_for_pi = 50e-9
    # Calculate the length of the pulse
    length = length_for_pi * (angle / np.pi)
    dsl.play(
        q.signals["drive"],
        amplitude=amplitude,
        phase=0.0,
        length=length,
        pulse=dsl.pulse_library.const(),
    )

Applying the decorator `qop.register` wraps our function `simple_rx` in a quantum operation and registers it with our current set of operations, `qop`.

We can confirm that it's registered by checking that its in our set of operations, or by looking it up as an attribute or element of our operations:

In [ ]:
"simple_rx" in qop

In [ ]:
qop.simple_rx

In [ ]:
qop["simple_rx"]

If an operation with the same name already exists it will be replaced.

Let's run our new operations and examine the section it produces:

In [ ]:
section = qop.simple_rx(qubits[0], np.pi)
print(section)

We can also create aliases for existing quantum operations that are already registered by assigning additional names for them:

In [ ]:
qop["rx_length"] = qop.simple_rx

In [ ]:
"rx_length" in qop

### Using omit_section

Let's imagine that we'd like to write an `x90_length` operation that calls our new `rx_length` but always specifies an angle of $\frac{\pi}{2}$. We can write this as:

In [ ]:
@qop.register
def x90_length(self, q):
    return self.rx_length(q, np.pi / 2)

However, when we call this we will have deeply nested sections and many signal lines reserved. This obscures the structure of our experiment:

In [ ]:
section = qop.x90_length(qubits[0])
print(section)

We can remove the extra section and signal reservations by call our inner operation using `.omit_section` instead:

In [ ]:
@qop.register
def x90_length(self, q):
    return self.rx_length.omit_section(q, np.pi / 2)

Note how much simpler the section structure looks now:

In [ ]:
section = qop.x90_length(qubits[0])
print(section)

### Replacing a quantum operation

To end off our look at quantum operations, let's replace the original `rx` gate with our own one and then use our existing experiment definition to produce a new experiment with the operation we've just written.

In [ ]:
qop["rx"] = qop.simple_rx  # replace the rx gate
exp = rotate_and_measure(qop, qubits[0], np.pi / 2)
print(exp)

Confirm that the generated experiment contains the new implementation of the RX gate.

### Setting section attributes

Sometimes an operation will need to set special section attributes such as `on_system_grid`.

This can be done by retrieving the current section and directly manipulating it.

To demonstrate, we'll create an operation whose section is required to be on the system grid:

In [ ]:
@qop.register
def op_on_system_grid(self, q):
    section = dsl.active_section()
    section.on_system_grid = True
    # ... play pulses, etc.

And then call it to confirm that the section has indeed been set to be on the grid:

In [ ]:
section = qop.op_on_system_grid(qubits[0])
section.on_system_grid

### Accessing experiment calibration

When a qubit experiment is created by the library its calibration is initialized from the qubits it operates on. Typically oscillator frequencies and other signal calibration are set.

Sometimes it may be useful for quantum operations to access or manipulate this configuration using `experiment_calibration` which returns the calibration set for the current experiment.

**Note**:

* The experiment calibration is only accessible if there is an experiment, so quantum operations that call `experiment_calibration` can only be called inside an experiment and will raise an exception otherwise.

* There is only a single experiment calibration per experiment, so if multiple quantum operations modify the same calibration items, only the last modification will be retained.

Here is how we define a quantum operation that accesses the calibration:

In [ ]:
@qop.register
def op_that_examines_signal_calibration(self, q):
    calibration = dsl.experiment_calibration()
    signal_calibration = calibration[q.signals["drive"]]
    # ... examine or set calibration, play pulses, etc, e.g.:
    signal_calibration.oscillator.frequency = 0.2121e9

To use it we will have to build an experiment. For now, just ignore the pieces we haven't covered. Writing a complete experiment will be covered shortly:

In [ ]:
@dsl.qubit_experiment
def exp_for_checking_op(qop, q):
    """Simple experiment to test the operation we've just written."""
    with dsl.acquire_loop_rt(count=1):
        qop.op_that_examines_signal_calibration(q)


exp = exp_for_checking_op(qop, qubits[0])
exp.get_calibration().calibration_items["/logical_signal_groups/q0/drive"]

Note above that the oscillator frequency has been set to the value we specified, `0.2121e9` Hz.